## Learning objectives
- Calculate metrics of a regression model
- Understand why two predictors together in the same model have different slopes than if they are in separate models.

In [14]:
## our previous code...
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_theme()
from scipy.stats import bernoulli, norm
fruit_length_geno = pd.read_table("../data/arabmagic/fruit_length_geno.csv",sep=",",index_col=0)

import statsmodels.api as sm

### MASC06116 model
to_model = pd.DataFrame({"fruit_length":fruit_length_geno['fruit_length'],
                         "MASC06116": (fruit_length_geno['MASC06116']=='A').astype(int),
                         "MASC02863": (fruit_length_geno['MASC02863']=='A').astype(int),
                         "MASC06116_dup": (fruit_length_geno['MASC06116']=='A').astype(int)})
                                                                                    ###^^or *2
to_model = to_model.loc[ pd.isnull(to_model).sum(axis=1)==0, :]

to_model = sm.add_constant(to_model)

print('MASC06116',to_model.shape)
X_marker = to_model.loc[:,['const','MASC06116']]
y_marker = to_model['fruit_length']
model_marker = sm.OLS(y_marker, X_marker).fit()
#model_marker.summary()


### height model
# to_model = pd.DataFrame({"fruit_length":fruit_length_geno['fruit_length'],
#                          "height": fruit_length_geno['height']})
#                                                                                     ###^^or *2
# to_model = to_model.loc[ pd.isnull(to_model).sum(axis=1)==0, :]

# to_model = sm.add_constant(to_model)
# print('height',to_model.shape)
# X_height = to_model.loc[:,['const','height']]
# y_height = to_model['fruit_length']
# model_height = sm.OLS(y_height, X_height).fit()
#model_height.summary()

MASC06116 (674, 5)


## 5. Linear regression Model metrics
A **metric** is a formula we calculate using the data + the model to figure out how well the model fits the data

**Question**: what metrics have we discussed so far? How can we decide which single predictor is more predictive of fruit_length?

### 5.1 Residuals

The difference between our predicted average value of fruit length and the actual value is called the **residual**. You can think of this as the **error** in our prediction... the bigger the difference, the worse we did.

**Exercise 5.1.1**: Get the predictions for `model_marker` and save it in a variable called `predicted values`. Calculate the residuals  of fruit length.  

You can also get this using the attribute `resid`:

In [ ]:
model_marker.resid

**Exercise 5.1.2**: Make a strip plot or violin plot of residuals for plants separated by the 2 values of genotypeNumber (refer to Seaborn docs [here](https://seaborn.pydata.org/generated/seaborn.catplot.html))

**Exercise 5.1.3**: Find out whether the error in our model gets worse for taller plants by plotting height versus residual. 

The **root mean squared error or RMSE** is a measure of how well your predictions fit the data. It is defined as:
1. square the residuals (*squared error*)
2. get the mean of the squared residuals (*mean squared error*)
3. take the square root of that (*root mean squared error*)

**Exercise 5.1.4**: Calculate Root Mean Squared Error using your predictions for the height model and for the MASC06116 model.

You can calculate root mean squared error using the `rmse` function, alongside your predicted values as follows:

In [ ]:
from statsmodels.tools.eval_measures import rmse
rmse(predicted_values, y)

### 5.2 R-squared

R-squared tells us the percent of variation in the fruit_length that is explained by the model, which we can see is 17% of the variation:

In [3]:
model_marker.rsquared

0.16990307104374025

In [4]:
fruit_length_geno['fruit_length'].var()

3.2207451545628083

**Exercise 5.2.1**: Predict the mean values based on only genotype and intercept and get the variance of these (using `var()`). Divide that by the variance of the fruit_lengths overall to calculate fraction of variance explained. Compare that to the `rsquared`.

**Exercise 5.2.2**: Is height a better predictor of fruit length? How can we use the summaries of the regression model to answer that question? Compare the models in 3 different metrics.

## 6 Regressions with more than 1 predictor
Linear regression uses the maximum likelihood method to build a model of the **dependent variable (aka response)** feature where instead of the mean being a constant number (like 12) the mean of the model depends on the **independent variable (aka predictor)** features (fruit length mean = 12 + 3 * genotypeNumber). The value we expect for the dependent variable *depends* on the value of the independent variables.

We've been just using the first marker in fruit_length_geno to predict fruit_length. Let's try a different marker.

In [3]:
fruit_length_geno.head()

,MASC06116,MASC02863,FES1_3177,RAX2_405,MASC05402,bolting_days,seed_weight,seed_area,ttl_seedspfruit,branches,height,pc_seeds_aborted,fruit_length
MAGIC.1,A,A,A,A,A,15.33,17.15,0.64,45.11,10.50,NaN,0.00,14.95
MAGIC.10,A,A,A,A,A,30.00,30.46,0.91,56.33,3.00,48.50,0.41,18.63
MAGIC.100,A,A,A,A,A,38.33,23.59,0.76,53.33,8.67,51.43,0.00,16.36
MAGIC.101,A,A,A,A,A,17.00,23.20,0.75,64.00,5.67,NaN,0.36,17.65
MAGIC.102,A,A,A,A,A,15.67,22.12,0.72,55.56,7.33,41.75,0.00,16.52


**Exercise 6.1**: Using a different marker to predict fruit length (Model 2) 
1. Use the marker MASC02863 as a predictor variable and do regression predicting fruit length (save this into a variable called `model2`.

In [11]:
X = to_model.loc[:, ['const', 'MASC02863']]
y = to_model['fruit_length']
model2 = sm.OLS(y, X).fit()

2. Write down the linear model including the coefficients of the model. 

3. Calculate the predicted mean fruit lengths and the RMSE

4. Is this marker a good predictor of fruit length? Why? Compare it to `model_marker` using at least 2 metrics. 

But there's no reason we have to pick one. We can use more than one by including it in the data frame `X`.

We will fit a linear model like this one:
    
${mean Fruit Length} = intercept + slope1 \times MASC06116 + slope2 \times MASC02863 $

(where the markers are converted into 0 and 1 as above)    

**Exercise 6.2**: Using both markers (Model 3).
1. Run the regression now including both MASC06116 and MASC02863 as predictors of fruit length  (save this into a variable called `model3`). How can we compare it to the other models? **We can use the data summary and evaluate the RMSE of the predictions**

In [8]:
X = to_model.loc[:, ['const', 'MASC02863', 'MASC06116']]
y = to_model['fruit_length']
model13 = sm.OLS(y, X).fit()

2. Write out the linear model that includes both markers, including the actual numbers of the parameters (rounded to 2 decimals).

In [10]:
display(model13.summary())
print(f'fruit_length = {model13.params[0]} + {model13.params[2]} * MASC06116 + {model13.params[1]} * MASC02863')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           fruit_length   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                     69.25
Date:                Wed, 30 Nov 2022   Prob (F-statistic):           4.56e-28
Time:                        13:31:34   Log-Likelihood:                -1286.8
No. Observations:                 674   AIC:                             2580.
Df Residuals:                     671   BIC:                             2593.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.2511      0.239     51.190      0.000      11.781      12.721
MASC02863      0.8216      0.838      0.981      0.327      -0.823       2.467
MASC06116      2.0770      0.822      2.527      0.012       0.463       3.691
==============================================================================
Omnibus:                        0.283   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.868   Jarque-Bera (JB):                0.197
Skew:                           0.036   Prob(JB):                        0.906
Kurtosis:                       3.044   Cond. No.                         30.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

fruit_length = 12.251128087925638 + 2.0769798674952584 * MASC06116 + 0.8215745638813132 * MASC02863


3. There are 4 possible genotypes considering these 2 markers, which is all combinations of A and B at the 2 markers. The values are below (converted to A=1, B=0) in the data frame `genoPairs`. Use the linear model in step 2 (or, if you prefer, the `predict` function) to predict mean fruit lengths for plants in the 4 categories. Add this prediction as a column to the data frame. 

In [12]:
geno_pairs = pd.DataFrame({'MASC06116':[0,0,1,1],
                           'MASC02863':[0,1,0,1]})
geno_pairs
geno_pairs = sm.add_constant(geno_pairs)
predictions = model13.predict(geno_pairs)
geno_pairs['model3_preds'] = predictions

predictions = model13.params[0] * geno_pairs['const'] + model_marker.params[1] * geno_pairs['MASC06116'] + model2.params[1] * geno_pairs['MASC02863']
geno_pairs['combined_preds'] = predictions

geno_pairs

,const,MASC06116,MASC02863,model3_preds,combined_preds
0,1.0,0,0,12.251128,12.251128
1,1.0,0,1,14.328108,15.095554
2,1.0,1,0,13.072703,15.098068
3,1.0,1,1,15.149683,17.942493


4. Compare the slope parameters for model_marker, model2, and model 3. What do you notice? Add another column to the genoPairs data frame that uses the slope parameters from Model 1 and Model 2, together to make predictions. What do you notice?

### 6.1 Collinearity of two independent variables
When two independent variables have almost the same values, they can't both have a high slope if they are together in the same model. Even if when they have a high slope when they are in separate models.

We can use the `crosstab` function to see what combinations of genotypes are present across the plants.

In [13]:
pd.crosstab(fruit_length_geno['MASC06116'],
            fruit_length_geno['MASC02863'])

MASC02863,A,B
MASC06116,,
A,627,1
B,3,45


**Exercise 6.1.1**: In your own words describe the relationship between the two markers by looking at this table. **When one marker is A, the other is A. They covary with one another.**

**Exercise 6.1.2**: Make another model `model4` that has 2 predictors, but now both predictors are the genotype at marker MASC06116 (so we have 2 identical independent variables). Compare the coefficients and r-squared between Model 4 and Model 1. What do you notice?

In [15]:
X = to_model.loc[:, ['const', 'MASC06116', 'MASC06116_dup']]
y = to_model['fruit_length']
model4 = sm.OLS(y, X).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           fruit_length   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                     137.5
Date:                Wed, 30 Nov 2022   Prob (F-statistic):           4.98e-29
Time:                        13:51:03   Log-Likelihood:                -1287.3
No. Observations:                 674   AIC:                             2579.
Df Residuals:                     672   BIC:                             2588.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            12.3014      0.234     52.625      0.000      11.842      12.760
MASC06116         1.4235      0.121     11.728      0.000       1.185       1.662
MASC06116_dup     1.4235      0.121     11.728      0.000       1.185       1.662
==============================================================================
Omnibus:                        0.247   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.884   Jarque-Bera (JB):                0.168
Skew:                           0.033   Prob(JB):                        0.919
Kurtosis:                       3.040   Cond. No.                     2.43e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.21e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""